In [1]:
import tensorflow as tf
import numpy as np

from sklearn.datasets import fetch_california_housing

In [2]:
housing = fetch_california_housing()

In [3]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [4]:
scaled = ss.fit_transform(housing.data)

In [5]:
scalled_housing_with_bias = np.c_[np.ones(shape=(housing.data.shape[0], 1)), scaled]

In [6]:
labels = housing.target.reshape(-1, 1)

In [7]:
scalled_housing_with_bias.shape

(20640, 9)

In [8]:
labels.shape

(20640, 1)

In [9]:
num_examples, num_features = scalled_housing_with_bias.shape

In [10]:
x = tf.placeholder(dtype=tf.float32, shape=(None, num_features))
y = tf.placeholder(dtype=tf.float32, shape=(None, 1))

In [11]:
batch_size = 100
n_epochs = 100
n_batches = int(np.ceil(num_examples/batch_size))

In [12]:
theta = tf.Variable(tf.random_normal(shape=(num_features, 1)), dtype=tf.float32)

In [13]:
y_pred = tf.matmul(x, theta)

In [14]:
error = y_pred - y

In [15]:
mse = tf.reduce_mean(tf.square(error))

In [16]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
training_op = optimizer.minimize(mse)

In [17]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(num_examples, size=batch_size)  # not shown
    X_batch = scalled_housing_with_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch


In [18]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [19]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            x_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
        
        if epoch % 10 == 0:
            save_path = saver.save(sess, "/tmp/model/my_model.ckpt")
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/model/my_model_final.ckpt")

In [20]:
best_theta

array([[ 2.0544672 ],
       [ 0.82970107],
       [ 0.10713524],
       [-0.31074822],
       [ 0.24564512],
       [-0.00222411],
       [-0.01061386],
       [-0.89121598],
       [-0.87524807]], dtype=float32)

In [21]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/model/my_model_final.ckpt")
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            x_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
        
        if epoch % 10 == 0:
            save_path = saver.save(sess, "/tmp/model/my_model1.ckpt")
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/model/my_model_final1.ckpt")

INFO:tensorflow:Restoring parameters from /tmp/model/my_model_final.ckpt
